In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/Capstone-Company-Based/dataset/train.csv', nrows=10000000)

In [ ]:
def final_clean(file, train = True):

    df_chunks = pd.read_csv(file, chunksize = 10000)

    chunk_list = []

    for chunk in df_chunks:
        
        #only taking the data with bookings
        if train:
            data = chunk.loc[chunk['hotel_country'] == 50]
        else:
            data = chunk

        # changing type to datetime
        data['date_time'] = pd.to_datetime(data['date_time'], format = '%Y-%m-%d').dt.normalize()
        data['srch_ci'] = pd.to_datetime(data['srch_ci'], format = '%Y-%m-%d', errors='coerce')
        data['srch_co'] = pd.to_datetime(data['srch_co'], format = '%Y-%m-%d', errors='coerce')
        
        #data cleaning for incorrectly input dates
        data['date_time'] = np.where(data['srch_ci'] < data['date_time'],
                                 data['srch_ci'],
                                 data['date_time'])

        data['srch_ci'], data['srch_co'] = np.where(data['srch_co'] < data['srch_ci'],
                                                            (data['srch_co'], data['srch_ci']),
                                                            (data['srch_ci'], data['srch_co'])
                                                           )
        
        #creating length of stay and days to checkin
        data['days_to_checkin'] = (data['srch_ci'] - data['date_time']).dt.days
        data['stay_length'] = (data['srch_co'] - data['srch_ci']).dt.days
        
        #creating length of stay and days to checkin
        data['month_of_booking'] = data['date_time'].dt.month
        data['month_of_staying'] = data['srch_ci'].dt.month
        
        col_to_drop = ['srch_ci', 
                       'srch_co', 
                       'site_name', 
                       'hotel_continent', 
                       'user_location_region', 
                       'srch_destination_type_id']

        data = data.drop(col_to_drop, axis=1)

        chunk_list.append(data)

    return pd.concat(chunk_list)

In [ ]:
df_concat = final_clean('/content/gdrive/MyDrive/Capstone-Company-Based/dataset/train.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas

In [ ]:
df_concat

,date_time,posa_continent,user_location_country,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,srch_adults_cnt,...,srch_destination_id,is_booking,cnt,hotel_country,hotel_market,hotel_cluster,days_to_checkin,stay_length,month_of_booking,month_of_staying
0,2014-08-11,3,66,48862,2234.2641,12,0,1,9,2,...,8250,0,3,50,628,1,16.0,4.0,8,8.0
1,2014-08-11,3,66,48862,2234.2641,12,0,1,9,2,...,8250,1,1,50,628,1,18.0,4.0,8,8.0
2,2014-08-11,3,66,48862,2234.2641,12,0,0,9,2,...,8250,0,1,50,628,1,18.0,4.0,8,8.0
3,2014-08-09,3,66,35390,913.1932,93,0,0,3,2,...,14984,0,1,50,1457,80,106.0,5.0,8,11.0
4,2014-08-09,3,66,35390,913.6259,93,0,0,3,2,...,14984,0,1,50,1457,21,106.0,5.0,8,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37670288,2014-09-02,3,66,26232,2348.4075,1198182,0,1,2,2,...,8855,0,1,50,213,26,75.0,6.0,9,11.0
37670289,2014-09-08,3,66,26232,679.6104,1198182,0,0,0,1,...,8281,0,1,50,663,9,42.0,5.0,9,10.0
37670290,2014-09-15,3,66,26232,668.1768,1198182,0,0,0,1,...,5620,0,1,50,663,94,0.0,1.0,9,9.0
37670291,2014-09-18,3,66,12565,106.4274,1198182,0,0,0,1,...,18811,0,1,50,592,42,0.0,1.0,9,9.0


In [ ]:
df_concat.isnull().sum()

date_time                          0
posa_continent                     0
user_location_country              0
user_location_city                 0
orig_destination_distance    3965230
user_id                            0
is_mobile                          0
is_package                         0
channel                            0
srch_adults_cnt                    0
srch_children_cnt                  0
srch_rm_cnt                        0
srch_destination_id                0
is_booking                         0
cnt                                0
hotel_country                      0
hotel_market                       0
hotel_cluster                      0
days_to_checkin                18897
stay_length                    18900
month_of_booking                   0
month_of_staying               18897
dtype: int64

In [ ]:
df_concat['days_to_checkin'].fillna(df_concat['days_to_checkin'].mode()[0], inplace=True)
df_concat['stay_length'].fillna(df_concat['stay_length'].mode()[0], inplace=True)
df_concat['month_of_staying'].fillna(df_concat['month_of_staying'].mode()[0], inplace=True)

In [ ]:
df_concat.to_csv('/content/gdrive/MyDrive/Capstone-Company-Based/dataset/train_clear.csv', index=False)